## Initialization

*Run once, upon starting a session.*

In [ ]:
!pip install git+https://github.com/ribqahisabsent/mc-diag-boat.git@main


### SCHEMATIC CODE ###

from litemapy import BlockState
from mc_diag_boat.vec2 import Vec2
import mc_diag_boat.schematic as sch
import mc_diag_boat.input as inp
import mc_diag_boat.report as rep


BLOCKS = [
    "minecraft:blue_ice",
    "minecraft:packed_ice",
    "minecraft:ice",
    "minecraft:stone",
    "minecraft:stone_button",
    "minecraft:stone_pressure_plate",
    "minecraft:lever",
    "minecraft:rail",
]


def get_boat_offsets(offset: Vec2[int]) -> list[Vec2[float]]:
    return [
        offset.project(Vec2.from_polar(1.0, angle))
        for angle in offset.angle().closest_boat_angle(4)
    ]


def choose_path_offset(
    origin: Vec2[int],
    offset: Vec2[int],
    boat_offsets: list[Vec2[float]],
) -> Vec2[float]:
    print(f"""
Offset: {offset}, Distance: {round(offset.length(), 2)}
Closest boat offsets:""")
    lines = rep.pretty_seqs([(
        index,
        ": destination:",
        (origin + boat_offset).round(),
        " error:",
        f"{round((boat_offset - offset).length(), 2)} blocks",
    ) for index, boat_offset in enumerate(boat_offsets)])
    for line in lines:
        print("   ", line)
    return boat_offsets[inp.loop_input(
        "\nEnter index of desired boat angle (default, 0): ",
        {index for index in range(len(boat_offsets))},
        default=0,
    )]


def choose_blocks() -> list[BlockState]:
    print("\nBlock options:")
    for index, block in enumerate(BLOCKS):
        print("   ", index + 1, ":", block)
    while True:
        indices = str(inp.loop_input(
            "\nEnter indices of desired blocks in order (default, 15 (blue ice topped with buttons)): ",
            int,
            default=15,
        ))
        try:
            if any(int(index) - 1 < 0 for index in indices):
                raise ValueError("Indices must be 1 or greater")
            return [
                BlockState(BLOCKS[int(index) - 1]).with_properties(face="floor", facing="north")
                for index in indices
            ]
        except Exception as e:
            print(f"{type(e).__name__}, {e}")
            continue


def create_schematic(
    origin: Vec2[int],
    path_offset: Vec2[float],
    blocks: list[BlockState],
    gap_size: int,
) -> None:
    schem_name = f"dbpath_{origin.dense_str()}_{(origin + path_offset.round()).dense_str()}"
    schem = sch.generate_schematic(path_offset, gap_size, blocks,schem_name)
    schem.save(schem_name + ".litematica")
    print("\nSaved schematic", schem_name)
    path_angle = path_offset.angle().closest_boat_angle()
    print(f"""
Boat placement angle range: {path_angle.boat_placement_range()}
    F3 angle while in boat: {round(path_angle, 1):.1f}""")


### PATTERN CODE ###

from mc_diag_boat.vec2 import Vec2
from mc_diag_boat.pattern import Pattern, PatternGenerator
import mc_diag_boat.optimization as opt
import mc_diag_boat.input as inp
import mc_diag_boat.report as rep


def get_boat_offsets(offset: Vec2[int]) -> list[Vec2[float]]:
    return [
        offset.project(Vec2.from_polar(1.0, angle))
        for angle in offset.angle().closest_boat_angle(4)
    ]


def get_patterns(offsets: list[Vec2[float]]) -> list[Pattern]:
    return [
        pattern
        for offset in offsets
        for pattern in PatternGenerator(offset).patterns
    ]


def get_pareto_patterns(
    offset: Vec2[int],
    patterns: list[Pattern],
) -> list[Pattern]:
    patterns_attrs = [
        (-(offset - pattern.target).length(), -pattern.deviation(), -len(pattern))
        for pattern in patterns
    ]
    pareto_patterns = [
        patterns[index]
        for index in opt.pareto_indices(patterns_attrs)
    ]
    return [
        pattern
        for index, pattern in enumerate(pareto_patterns)
        if not any(pattern == other for other in pareto_patterns[:index])
    ]


def choose_pattern(
    origin: Vec2[int],
    offset: Vec2[int],
    patterns: list[Pattern],
) -> Pattern:
    sorted_patterns = sorted(patterns, key=lambda p: p.deviation())
    print(f"""
Offset: {offset}, Distance: {round(offset.length(), 2)}
Patterns:""")
    lines = rep.pretty_seqs([(
        index,
        ": destination:",
        (origin + pattern.target).round(),
        " dest_error:",
        f"{round((offset - pattern.target).length(), 2)} blocks",
        " n_blocks:",
        len(pattern) - 1,
        " travel_error:",
        f"{round(pattern.deviation(), 2)} blocks",
    ) for index, pattern in enumerate(sorted_patterns)])
    for line in lines:
        print("   ", line)
    choice = inp.loop_input(
        "\nEnter index of desired pattern (default, 0): ",
        {index for index in range(len(sorted_patterns))},
        default=0,
    )
    return sorted_patterns[choice]


def display_pattern(origin: Vec2[int], pattern: Pattern) -> None:
    pattern_name = f"dbpatt_{origin.dense_str()}_{(origin + pattern.target.round()).dense_str()}"
    fig, plt = pattern.plot()
    fig.savefig(pattern_name)
    print("\nSaved pattern", pattern_name)
    boat_angle = pattern.target.angle().closest_boat_angle()
    print(f"""
Boat placement angle range: {boat_angle.boat_placement_range()}
    F3 angle while in boat: {round(boat_angle, 1):.1f}""")
    plt.show()


## Schematic generator

*For use with Litematica.*

In [ ]:
origin = inp.vec2_input("Enter origin", int)
destination = inp.vec2_input("Enter destination", int)
offset = destination - origin
boat_offsets = get_boat_offsets(offset)
path_offset = choose_path_offset(origin, offset, boat_offsets)
gap_size = inp.loop_input(
    "\nEnter gap size (default, 0): ",
    {n for n in range(255)},
    default=0,
)
blocks = choose_blocks()
create_schematic(origin, path_offset, blocks, gap_size)


## Pattern generator

*Modless path building guide.*

In [ ]:
origin = inp.vec2_input("Enter origin", int)
destination = inp.vec2_input("Enter destination", int)
offset = destination - origin
boat_offsets = get_boat_offsets(offset)
patterns = get_patterns(boat_offsets)
pareto_patterns = get_pareto_patterns(offset, patterns)
pattern = choose_pattern(origin, offset, pareto_patterns)
display_pattern(origin, pattern)
